In [1]:
from IPython.display import display
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook
from capstone_project import preprocessor as pre

tqdm_notebook().pandas(desc="Progress:")

In [2]:
data = pd.read_csv("/home/andre/Documents/mooc/udacity/machinelearning/capstone_project/data/questions.csv")

In [3]:
display(data.head())

print "Rows in data: {}".format(len(data))
print "Ratio of duplicates: {:.2f}".format(float(len(data[data.is_duplicate == 1]))/len(data))

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


Rows in data: 404351
Ratio of duplicates: 0.37


In [4]:
labels = data.iloc[:, -1].values
# Stratified splitting is enabled because the data is somewhat imbalanced(~37% are duplicates)
train_val_data, test_data = train_test_split(data, test_size=0.1, stratify=labels, random_state=42)

In [5]:
display(train_val_data.head(1))
display(test_data.head(1))

print "Rows in test set: {} and train_val set: {}".format(len(test_data), len(train_val_data))
print "Ratio: {:.2f}".format(float(len(test_data))/len(train_val_data))
print "Ratio of duplicates test: {:.2f}".format(float(len(test_data[test_data.is_duplicate == 1]))/len(test_data))
print "Ratio of duplicates train: {:.2f}".format(float(len(train_val_data[train_val_data.is_duplicate == 1]))/len(train_val_data))

,id,qid1,qid2,question1,question2,is_duplicate
218104,218104,430027,430028,How do I build a self confidence?,How can I raise my self esteem?,1


,id,qid1,qid2,question1,question2,is_duplicate
113079,113079,224193,224194,What does horny goat weed do?,What is horny goat weed?,1


Rows in test set: 40436 and train set: 363915
Ratio: 0.11
Ratio of duplicates test: 0.37
Ratio of duplicates train: 0.37


In [6]:
#split one more time to create validation set because neural nets train to long to use cross validation
labels = train_val_data.iloc[:, -1].values
train_data, val_data = train_test_split(train_val_data, test_size=0.2, stratify=labels, random_state=75293)

In [ ]:
print "Rows in train set: {} and val set: {}".format(len(train_data), len(val_data))

In [7]:
# Data is tokenized here and not in the analysis pipeline because it takes some time and only needs to be done once.
train_data = train_data.fillna("")  # Two questions have empty fields
train_data["q1_tokens"] = train_data["question1"].progress_apply(pre.tokenize)
train_data["q2_tokens"] = train_data["question2"].progress_apply(pre.tokenize)

val_data = val_data.fillna("")  # Two questions have empty fields
val_data["q1_tokens"] = val_data["question1"].progress_apply(pre.tokenize)
val_data["q2_tokens"] = val_data["question2"].progress_apply(pre.tokenize)

test_data = test_data.fillna("")  # Two questions have empty fields
test_data["q1_tokens"] = test_data["question1"].progress_apply(pre.tokenize)
test_data["q2_tokens"] = test_data["question2"].progress_apply(pre.tokenize)


In [8]:
output_directory = "../output/data/"
prefix = "tokenized_"

pre.save_pickle(train_data, output_directory, prefix+"train_data.pkl") 
pre.save_pickle(val_data, output_directory, prefix+"val_data.pkl")
pre.save_pickle(test_data, output_directory, prefix+"test_data.pkl") 

In [9]:
# Create cross validation folds to tune hypermparameters of "classic" models such as logistic regression and boosted trees
# TODO delete not needed any longer due to validation set
#skf = StratifiedKFold(n_splits=3, random_state=12574, shuffle=True)
#output_directory = "../output/models/"
#pre.save_as_pickle(skf, output_directory, "kfolds.pkl") 